In [ ]:
!pip install torchcrepe
!wget https://github.com/ursinus-cs372-s2023/HW6_StringAlong/raw/main/adele.wav

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import torchcrepe # https://github.com/maxrmorrison/torchcrepe
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from scipy import signal
import librosa
import time
from scipy.io import wavfile

device = 'cuda:0'

## Sonify Adele

In [ ]:
sr = 16000
hop_length = 64
x, _ = librosa.load("adele.wav", sr=sr)


fmin = 50
fmax = 2000
pitch = torchcrepe.predict(torch.from_numpy(x).view((1, x.size)),sr,hop_length,fmin,fmax,'full',
                           batch_size=2048,device=device).flatten()

In [ ]:
## TODO: Sonify pitch
plt.plot(pitch)

## Sonify Harmonics

In [ ]:
n_harmonics = 10
C = torch.ones(1, sr, n_harmonics)
freqs = torch.arange(1, n_harmonics+1).view(1, n_harmonics)
times = torch.arange(sr).view(sr, 1)
decay = np.exp(-freqs*times/sr)
C = C*decay
plt.imshow(C[0, :, :].T, aspect='auto', cmap='magma')
plt.ylim([0, n_harmonics-1])

In [ ]:
## TODO: Sonify harmonics over time using torch.cos
## Do it first using a loop if you must, then try it without a loop

## Applying Reverb

In [ ]:
def fftconvolve(x, h):
    """
    Perform a fast convolution of two tensors across their last axis
    by using the FFT. Since the DFT assumes circularity, zeropad them 
    appropriately before doing the FFT and slice them down afterwards
    
    The last axis should be the same length, and the result will 
    be equivalent to np.convolve's 'same'
    
    Parameters
    ----------
    x: torch.tensor
        First tensor
    h: torch.tensor
        Second tensor
    
    Returns
    -------
    Tensor resulting from the convolution of x and y across their last axis
    """
    N = max(x.shape[-1], h.shape[-1])
    if x.shape[-1] != h.shape[-1]:
        # Zeropad so they're equal
        if x.shape[-1] < N:
            x = nn.functional.pad(x, (0, N-x.shape[-1]))
        if h.shape[-1] < N:
            h = nn.functional.pad(h, (0, N-h.shape[-1]))
    x = nn.functional.pad(x, (0, N))
    h = nn.functional.pad(h, (N, 0))
    X = torch.fft.rfft(x)
    H = torch.fft.rfft(h)
    y = torch.fft.irfft(X*H)
    return y[..., N::]

In [ ]:
x = torch.from_numpy(x)

In [ ]:
xlen = len(x)

x = x.view(1, xlen, 1)

h = torch.exp(-20*torch.arange(sr)/sr)